In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
last_week = pd.read_excel(r"C:\Users\pa173\Documents\SQL Excel Extracts\Weekly Producer\January\Jan 15 - Jan 21 2023.xlsx", sheet='Export Worksheet', index=False)

In [3]:
all_parent = pd.read_excel(r"C:\Users\pa173\Documents\Communications\All Producer with Parent Rollup.xlsx", index=False)

In [4]:
rank_list = pd.read_excel(r"C:\Users\pa173\Documents\Communications\Wkly Working - Rank and List.xlsx", index=False)

In [5]:
last_report = pd.read_excel(r"C:\Users\pa173\Documents\Communications\Weekly NB and RW Apps and FQ.xlsx", sheet_name='By Producer Weekly Counts', index=False, skiprows= 9)
last_report = last_report.drop(index=0)

In [6]:
nb_rw_pivot = pd.read_excel(r"C:\Users\pa173\Documents\Communications\Weekly NB and RW Apps and FQ.xlsx", sheet_name='Pivot', index=False, skiprows= 6)

In [7]:
def modify_cols(df):
    df.columns=df.columns.str.lower()
    df.columns=df.columns.str.strip()
    df.columns=df.columns.str.replace(" ", "_")
    return df


In [8]:
last_week=modify_cols(last_week)
all_parent=modify_cols(all_parent)
rank_list=modify_cols(rank_list)
last_report=modify_cols(last_report)
nb_rw_pivot=modify_cols(nb_rw_pivot)

merging last weeks data with all producers

In [9]:
parent_week=pd.merge(left=last_week, right=all_parent, left_on='sub_business_unit_number', right_on='business_unit_number', how='left')

In [10]:
parent_week.shape

(471, 15)

In [11]:
producer_pivot=parent_week.pivot_table(index=['parent_name'], values=['count(*)'], aggfunc='sum')
#producer_pivot = parent_week.pivot_table(index='parent_name', values='count(*)', columns=['parent_name','count(*)'], aggfunc='sum').sort_values(['count(*)'], ascending=False)

In [12]:
producer_pivot.shape

(457, 1)

In [13]:
rank_list.shape

(1012, 3)

let's merge our producer pivot table with the weekly rank

In [14]:
producer_rank=pd.merge(left=producer_pivot, right=rank_list, left_on='parent_name', right_on='producer', how='left')

In [15]:
producer_rank.shape

(457, 4)

cleaning nb and rw pivots

In [16]:
nb_pivot = nb_rw_pivot[['row_labels', 'unnamed:_8']]
nb_pivot=nb_pivot.rename(columns={'row_labels':'nb_labels','unnamed:_8':'nb_total'})
nb_pivot=nb_pivot.drop(nb_pivot.tail(1).index)

In [17]:
nb_pivot.tail()

,nb_labels,nb_total
199,LD BROOKS FINANCIAL & INS GRP,1
200,BROWNSTONE TAX & FINANCIAL SVC,1
201,"FREEDOM SERVICE GROUP, LLC",1
202,THOMAS J CORBITT,1
203,G.A. BONDON INSURANCE SVS LLC,1


In [18]:
rw_pivot = nb_rw_pivot[['row_labels.1', 'unnamed:_19']]
rw_pivot=rw_pivot.rename(columns={'row_labels.1':'rw_labels','unnamed:_19':'rw_total'})
rw_pivot=rw_pivot.dropna()
rw_pivot=rw_pivot.drop(rw_pivot.tail(1).index)

In [19]:
rw_pivot.tail()

,rw_labels,rw_total
98,ERICK A. REQUENO BONILLA,0.0
99,"GUZMAN INSURNCE SERVICE, INC",1.0
100,DALE GANLEY CLABAUGH,1.0
101,RAYNOLD W MENSAH,1.0
102,MY NATL TAX & INS SRVCS INC,1.0


In [20]:
nb_pivot.shape, rw_pivot.shape

((204, 2), (103, 2))

clean last weeks report

In [21]:
last_report = last_report.iloc[:, 1:17]
last_report.iloc[:,4:16] = last_report.iloc[:, 4:16].apply(pd.to_numeric)
last_report= last_report.rename(columns = {'weekly_counts_by_week': 'producer'})

merge parent pivot with last weeks report

In [22]:
new_report=pd.merge(last_report, producer_rank, left_on='producer', right_on='producer', how='left')

In [23]:
new_report.shape

(1015, 19)

merge new report with nb and rw tables

In [24]:
new_report=pd.merge(new_report, nb_pivot, left_on='producer', right_on='nb_labels', how='left')

In [25]:
new_report.shape

(1015, 21)

In [26]:
new_report=pd.merge(new_report, rw_pivot, left_on='producer', right_on='rw_labels', how='left')

In [27]:
new_report.shape

(1015, 23)

In [28]:
new_report=new_report.fillna(0)

In [29]:
new_report['incl']=new_report['incl'].replace(0,'N')

converting incl column as category since it's a boolean column

In [30]:
new_report['incl']=new_report['incl'].astype('category')

updating file with this weeks data

In [31]:
true_count = (new_report['count(*)'].isin(new_report['quotes'])).sum()
false_count = (new_report['count(*)'].isin(new_report['quotes'])==False).sum()
print("Number of true values:", true_count)
print("Number of false values:", false_count)


Number of true values: 1015
Number of false values: 0


In [32]:
new_report['updated_quotes']=new_report['count(*)']
new_report['updated_nb_apps']=new_report['nb_total']
new_report['updated_rw_apps']=new_report['rw_total']

In [33]:
new_report['change_quotes']=new_report['count(*)'] - new_report['quotes.1']
new_report['change_nb_apps']=new_report['nb_total'] - new_report['nb_apps.1']
new_report['change_rw_apps']=new_report['rw_total'] - new_report['rw_apps.1']

updating only the columns we need

In [34]:
#new_report=new_report[['producer', 'change_quotes', 'change_nb_apps', 'change_rw_apps', 
#        'updated_quotes', 'updated_nb_apps', 'updated_rw_apps',
#       'quotes', 'nb_apps', 'rw_apps', 'quotes.1', 'nb_apps.1', 'rw_apps.1',
#       'quotes.2', 'nb_apps.2', 'rw_apps.2', 'quotes.3']]

In [35]:
new_report.columns

Index(['producer', 'change_quotes', 'change_nb_apps', 'change_rw_apps',
       'quotes', 'nb_apps', 'rw_apps', 'quotes.1', 'nb_apps.1', 'rw_apps.1',
       'quotes.2', 'nb_apps.2', 'rw_apps.2', 'quotes.3', 'nb_apps.3',
       'rw_apps.3', 'count(*)', 'rank', 'incl', 'nb_labels', 'nb_total',
       'rw_labels', 'rw_total', 'updated_quotes', 'updated_nb_apps',
       'updated_rw_apps'],
      dtype='object')

In [36]:
new_report=new_report.sort_values(by='quotes', ascending=False)

In [37]:
new_report['pct_change_quotes']=(new_report['quotes']/new_report['quotes.1']*100.0) - 100
new_report['pct_change_quotes']=new_report['pct_change_quotes'].round(2)
new_report['pct_change_quotes']=new_report['pct_change_quotes'].replace([np.inf, -np.inf, np.nan], 0)
new_report['pct_change_quotes']=new_report['pct_change_quotes'].astype(str) + '%'

In [38]:
new_report['pct_change_nb_apps']=(new_report['nb_apps']/new_report['nb_apps.1']*100.0) - 100
new_report['pct_change_nb_apps']=new_report['pct_change_nb_apps'].round(2)
new_report['pct_change_nb_apps']=new_report['pct_change_nb_apps'].replace([np.inf, -np.inf, np.nan], 0)
new_report['pct_change_nb_apps']=new_report['pct_change_nb_apps'].astype(str) + '%'

In [39]:
new_report['pct_change_rw_apps']=(new_report['rw_apps']/new_report['rw_apps.1']*100.0) - 100
new_report['pct_change_rw_apps']=new_report['pct_change_rw_apps'].round(2)
new_report['pct_change_rw_apps']=new_report['pct_change_rw_apps'].replace([np.inf, -np.inf, np.nan], 0)
new_report['pct_change_rw_apps']=new_report['pct_change_rw_apps'].astype(str) + '%'

In [40]:
new_report=new_report[['producer', 'pct_change_quotes', 'pct_change_nb_apps', 'pct_change_rw_apps',
       'change_quotes', 'change_nb_apps', 'change_rw_apps',
       'quotes', 'nb_apps', 'rw_apps', 'quotes.1', 'nb_apps.1', 'rw_apps.1',
       'quotes.2', 'nb_apps.2', 'rw_apps.2', 'quotes.3', 'nb_apps.3',
       'rw_apps.3', 'count(*)', 'rank', 'incl', 'nb_labels', 'nb_total',
       'rw_labels', 'rw_total', 'updated_quotes', 'updated_nb_apps',
       'updated_rw_apps']]

In [41]:
new_report.head()

,producer,pct_change_quotes,pct_change_nb_apps,pct_change_rw_apps,change_quotes,change_nb_apps,change_rw_apps,quotes,nb_apps,rw_apps,...,count(*),rank,incl,nb_labels,nb_total,rw_labels,rw_total,updated_quotes,updated_nb_apps,updated_rw_apps
0,ROADWAY INSURANCE INC #1,-8.93%,-11.76%,14.29%,-46.0,-8.0,6.0,469,60.0,48.0,...,469.0,1.0,Y,ROADWAY INSURANCE INC #1,60.0,ROADWAY INSURANCE INC #1,48.0,469.0,60.0,48.0
1,EZ INSURANCE GROUP LLC,-11.52%,0.0%,-5.56%,-47.0,0.0,-1.0,361,36.0,17.0,...,361.0,2.0,Y,EZ INSURANCE GROUP LLC,36.0,EZ INSURANCE GROUP LLC,17.0,361.0,36.0,17.0
4,JAMES BROWN INS AGENCY INC,33.18%,-6.9%,37.5%,74.0,-2.0,3.0,297,27.0,11.0,...,297.0,5.0,Y,JAMES BROWN INS AGENCY INC,27.0,JAMES BROWN INS AGENCY INC,11.0,297.0,27.0,11.0
3,PINNING INS AGENCY INC #1,8.45%,0.0%,-54.55%,18.0,0.0,-6.0,231,27.0,5.0,...,231.0,4.0,Y,PINNING INS AGENCY INC #1,27.0,PINNING INS AGENCY INC #1,5.0,231.0,27.0,5.0
5,RAMIREZ INSURANCE AGENCY INC,-8.99%,15.79%,100.0%,-17.0,3.0,3.0,172,22.0,6.0,...,172.0,6.0,Y,RAMIREZ INSURANCE AGENCY INC,22.0,RAMIREZ INSURANCE AGENCY INC,6.0,172.0,22.0,6.0


In [42]:
new_report.to_excel(r"C:\Users\pa173\Documents\Python\Weekly Report\new_report.xlsx", index=False)

In [43]:
new_report.columns

Index(['producer', 'pct_change_quotes', 'pct_change_nb_apps',
       'pct_change_rw_apps', 'change_quotes', 'change_nb_apps',
       'change_rw_apps', 'quotes', 'nb_apps', 'rw_apps', 'quotes.1',
       'nb_apps.1', 'rw_apps.1', 'quotes.2', 'nb_apps.2', 'rw_apps.2',
       'quotes.3', 'nb_apps.3', 'rw_apps.3', 'count(*)', 'rank', 'incl',
       'nb_labels', 'nb_total', 'rw_labels', 'rw_total', 'updated_quotes',
       'updated_nb_apps', 'updated_rw_apps'],
      dtype='object')

In [44]:
current_weekly_quotes=new_report['quotes'].sum()
current_weekly_nb_apps=new_report['nb_apps'].sum()
current_weekly_rw_apps=new_report['rw_apps'].sum()

In [50]:
current_weekly_quotes, current_weekly_nb_apps, current_weekly_rw_apps

(5958, 646.0, 257.0)

In [45]:
last_weekly_quotes=new_report['quotes.1'].sum()
last_weekly_nb_apps=new_report['nb_apps.1'].sum()
last_weekly_rw_apps=new_report['rw_apps.1'].sum()

In [49]:
last_weekly_quotes, last_weekly_nb_apps, last_weekly_rw_apps

(5892.0, 644.0, 254.0)

In [46]:
total_fq_change=current_weekly_quotes-last_weekly_quotes
total_nb_change=current_weekly_nb_apps-last_weekly_nb_apps
total_rw_change=current_weekly_rw_apps-last_weekly_rw_apps

In [48]:
total_fq_change, total_nb_change, total_rw_change

(66.0, 2.0, 3.0)

In [53]:
total_app_change=(current_weekly_nb_apps+current_weekly_rw_apps)-(last_weekly_nb_apps+last_weekly_rw_apps)
total_app_change

5.0